In [1]:
# !pip3 install pytorch-msssim
# !pip3 install piqa

import torch
import torch.nn as nn
import torch.nn.functional as Fo
import torch.optim as optim
import zlib
import matplotlib.pyplot as plt
import numpy as np
from pytorch_msssim import SSIM
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import save_image
from PIL import Image
import warnings
import os

base_dir = os.getcwd()

<h1>Environment setup</h1>

In [2]:
import numpy as np
from scipy.stats import gamma
import matplotlib.pyplot as plt

# Network parameters
NUM_EDGE_SERVERS = 3
PACKET_SIZE = 1500  # bytes
UPDATE_FREQUENCY = 10  # Hz (updates per second)

# Bandwidth parameters (Gamma distribution)
BANDWIDTH_ALPHA = 5
BANDWIDTH_BETA = 10  # Mbps

# delays ( ms )
PROPAGATION_DELAY = 0.5

# Current Generation hardware
MODEL_512_CURRENT_GENERATION_MIN_DELAY = 5.6
MODEL_512_CURRENT_GENERATION_MAX_DELAY = 24.8

MODEL_256_CURRENT_GENERATION_MIN_DELAY = 4.6
MODEL_256_CURRENT_GENERATION_MAX_DELAY = 22.6

# Next Generation hardware
MODEL_512_NEXT_GENERATION_MIN_DELAY = 1.5
MODEL_512_NEXT_GENERATION_MAX_DELAY = 6.8

MODEL_256_NEXT_GENERATION_MIN_DELAY = 1.2
MODEL_256_NEXT_GENERATION_MAX_DELAY = 5.8


COMPRESSION_DELAY = 0 # Almost zero
DECOMPRESSION_DELAY = 0 # Almost zero

# Edge server parameters
SERVICE_RATE = 100  # tasks per second
CPU_CAPACITY = 1e12 # FLOPs per second
ALPHA = 1e-9 # Scaling factor for processing time
MAX_LOAD = 0.9

# Offloading trigger
OFFLOADING_THRESHOLD = 50 # ms

#Simulation time
SIMULATION_TIME = 10 #seconds
TIME_STEP = 1/UPDATE_FREQUENCY
NUM_QUANTIZATION_LEVELS = 256

LATENT_SIZE = 512  # Replace with your actual latent size
FIXED_INPUT_SIZE = 256

In [6]:
import numpy as np
from collections import deque
import random

class OffloadingAgent:
    def __init__(self, num_servers):
        self.num_servers = num_servers
        self.action_size = num_servers  # Removed local processing option

        self.state_size = num_servers * 3 + 3  # [bandwidth, latency, loss] + vehicle state
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995

        self.memory = deque(maxlen=1000)
        self.policy_model = None  # Placeholder for the trained model

    def get_state(self, network_stats, vehicle_state, server_loads):
        state = []
        for i in range(self.num_servers):
            state.extend([
                network_stats[i]['bandwidth'],
                network_stats[i]['latency'],
                network_stats[i]['packet_loss']
            ])
        state.extend([
            vehicle_state['battery'],
            vehicle_state['cpu_load'],
            vehicle_state['gpu_load']
        ])
        state.extend(server_loads)
        return np.array(state, dtype=np.float32)

    def act(self, state):
        """Epsilon-greedy action selection, excluding local processing"""
        if random.random() <= self.epsilon:
            return random.randint(0, self.action_size - 1)

        # Use heuristic if no model is trained
        if self.policy_model is None:
            network_stats = state[:self.num_servers * 3].reshape(self.num_servers, 3)
            latencies = network_stats[:, 1]  # latency is second entry
            return np.argmin(latencies)  # always choose best edge server

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def calculate_reward(self, latency, battery_consumption, cpu_load, gpu_load):
        r_latency = -latency
        r_resource = -(battery_consumption + cpu_load + gpu_load)
        return 0.7 * r_latency + 0.3 * r_resource

In [7]:
import random

def simulate_bandwidth():
    """Simulates bandwidth using a Gamma distribution."""
    bandwidth = gamma.rvs(BANDWIDTH_ALPHA, scale=BANDWIDTH_BETA)
    return bandwidth # in Mbps

def calculate_latency(bandwidth, queue_length, service_rate, compressed_data_size=25, gen='current', model_kind='512'): 
    """Calculates one-way communication latency with more realistic queueing"""
    # Transmission delay (same as before)
    transmission_delay = compressed_data_size / (bandwidth * 1e6) * 1000  # ms
    
    # More realistic queuing delay (M/M/1 model)
    arrival_rate = queue_length / TIME_STEP if TIME_STEP > 0 else 0
    utilization = arrival_rate / service_rate
    
    if utilization >= 1:
        queuing_delay = float('inf')
    else:
        queuing_delay = (utilization / (service_rate * (1 - utilization))) * 1000  # ms

    # print(f'queue delay: {queuing_delay}')
    
    # [Rest of your existing latency calculation...]
    if gen == 'current':
        if model_kind == '512':
            models_delay = random.uniform(MODEL_512_CURRENT_GENERATION_MIN_DELAY, MODEL_512_CURRENT_GENERATION_MAX_DELAY)
        else:
            models_delay = random.uniform(MODEL_256_CURRENT_GENERATION_MIN_DELAY, MODEL_256_CURRENT_GENERATION_MAX_DELAY)
    else:
        if model_kind == '512':
            models_delay = random.uniform(MODEL_512_NEXT_GENERATION_MIN_DELAY, MODEL_512_NEXT_GENERATION_MAX_DELAY)
        else:
            models_delay = random.uniform(MODEL_256_NEXT_GENERATION_MIN_DELAY, MODEL_256_NEXT_GENERATION_MAX_DELAY)

    # Total latency
    total_latency = PROPAGATION_DELAY + queuing_delay + transmission_delay + models_delay
    return total_latency

In [8]:
def calculate_server_load(queue_length, service_rate, time_step):
    """Calculate server load based on queue length and service rate"""
    if service_rate * time_step == 0:
        return 0.0
    return min(queue_length / (service_rate * time_step), MAX_LOAD)

<h1>Standard simulation</h1>

In [9]:
from PIL import Image
import torchvision.transforms as transforms

def main(compressed_data_size: int, gen: str, model_kind: str):
    print(f"\n--- Simulation: {gen.title()} Gen | Model {model_kind} | Size: {compressed_data_size}KB ---")
    
    # Initialize with zero load and empty queues
    server_loads = [0.0 for _ in range(NUM_EDGE_SERVERS)]
    queue_length = [0.0 for _ in range(NUM_EDGE_SERVERS)]  # Using float for more precise calculations

    # Create RL Agent
    agent = OffloadingAgent(NUM_EDGE_SERVERS)

    # Dummy vehicle state
    vehicle_state = {
        'battery': 0.8,
        'cpu_load': 0.3,
        'gpu_load': 0.2
    }

    n_steps = int(SIMULATION_TIME * UPDATE_FREQUENCY)

    for step in range(n_steps):
        # 1. Simulate Network Conditions
        bandwidths = [simulate_bandwidth() for _ in range(NUM_EDGE_SERVERS)]
        network_stats = []
        for i in range(NUM_EDGE_SERVERS):
            network_stats.append({
                'bandwidth': bandwidths[i],
                'latency': 0,  # Will be filled below
                'packet_loss': np.random.uniform(0.0, 0.1)
            })

        # 2. Calculate Latency to each server
        latencies = []
        for i in range(NUM_EDGE_SERVERS):
            latency = calculate_latency(
                bandwidths[i], queue_length[i], SERVICE_RATE, compressed_data_size, gen, model_kind
            )
            latencies.append(latency)
            network_stats[i]['latency'] = latency

        # 3. Build state and choose action
        state = agent.get_state(network_stats, vehicle_state, server_loads)
        action = agent.act(state)

        # 4. Display current step information
        print(f"\nStep: {step}")
        print(f"  Bandwidths: {[f'{bw:.2f}' for bw in bandwidths]} Mbps")
        print(f"  Latencies:  {[f'{lat:.2f}' for lat in latencies]} ms")
        print(f"  Server Loads: {[f'{load:.2f}' for load in server_loads]}")
        print(f"  Queue Lengths: {[f'{q:.1f}' for q in queue_length]}")

        # 5. Show selected action
        if action == NUM_EDGE_SERVERS:
            print(f"  ➤ Agent Action: LOCAL PROCESSING")
        else:
            print(f"  ➤ Agent Action: OFFLOAD to Edge Server {action}")

        # 6. Simulate task arrivals and processing
        for i in range(NUM_EDGE_SERVERS):
            new_tasks = np.random.poisson(lam=20 * TIME_STEP)
            queue_length[i] += new_tasks

            processed_tasks = min(queue_length[i], SERVICE_RATE * TIME_STEP)
            queue_length[i] -= processed_tasks

            server_loads[i] = calculate_server_load(queue_length[i], SERVICE_RATE, TIME_STEP)

        # Small noise to simulate load change
        server_loads = [
            min(max(0, load + np.random.uniform(-0.02, 0.02)), MAX_LOAD)
            for load in server_loads
        ]

<h1>512x512 Output Model Simulation With Current Generation Hardware</h1>

In [10]:
main(125, 'current', '512')


--- Simulation: Current Gen | Model 512 | Size: 125KB ---

Step: 0
  Bandwidths: ['24.90', '91.08', '76.30'] Mbps
  Latencies:  ['22.61', '15.58', '15.44'] ms
  Server Loads: ['0.00', '0.00', '0.00']
  Queue Lengths: ['0.0', '0.0', '0.0']
  ➤ Agent Action: OFFLOAD to Edge Server 1

Step: 1
  Bandwidths: ['31.50', '32.55', '22.52'] Mbps
  Latencies:  ['20.81', '13.32', '19.81'] ms
  Server Loads: ['0.00', '0.00', '0.01']
  Queue Lengths: ['0.0', '0.0', '0.0']
  ➤ Agent Action: OFFLOAD to Edge Server 0

Step: 2
  Bandwidths: ['31.46', '28.89', '76.32'] Mbps
  Latencies:  ['7.64', '20.92', '12.48'] ms
  Server Loads: ['0.01', '0.00', '0.02']
  Queue Lengths: ['0.0', '0.0', '0.0']
  ➤ Agent Action: OFFLOAD to Edge Server 1

Step: 3
  Bandwidths: ['33.63', '47.47', '64.28'] Mbps
  Latencies:  ['17.96', '21.45', '23.70'] ms
  Server Loads: ['0.00', '0.00', '0.00']
  Queue Lengths: ['0.0', '0.0', '0.0']
  ➤ Agent Action: OFFLOAD to Edge Server 0

Step: 4
  Bandwidths: ['124.81', '26.07', '16

<h1>512x512 Output Model Simulation With Next Generation Hardware</h1>

In [11]:
main(125, 'next', '512')


--- Simulation: Next Gen | Model 512 | Size: 125KB ---

Step: 0
  Bandwidths: ['33.29', '46.87', '51.88'] Mbps
  Latencies:  ['6.91', '3.75', '6.28'] ms
  Server Loads: ['0.00', '0.00', '0.00']
  Queue Lengths: ['0.0', '0.0', '0.0']
  ➤ Agent Action: OFFLOAD to Edge Server 1

Step: 1
  Bandwidths: ['68.30', '45.58', '61.39'] Mbps
  Latencies:  ['7.25', '2.02', '6.91'] ms
  Server Loads: ['0.02', '0.02', '0.00']
  Queue Lengths: ['0.0', '0.0', '0.0']
  ➤ Agent Action: OFFLOAD to Edge Server 2

Step: 2
  Bandwidths: ['59.87', '26.79', '53.22'] Mbps
  Latencies:  ['3.73', '5.21', '6.60'] ms
  Server Loads: ['0.02', '0.00', '0.01']
  Queue Lengths: ['0.0', '0.0', '0.0']
  ➤ Agent Action: OFFLOAD to Edge Server 0

Step: 3
  Bandwidths: ['151.32', '33.30', '51.22'] Mbps
  Latencies:  ['6.10', '5.73', '5.88'] ms
  Server Loads: ['0.00', '0.01', '0.00']
  Queue Lengths: ['0.0', '0.0', '0.0']
  ➤ Agent Action: OFFLOAD to Edge Server 1

Step: 4
  Bandwidths: ['67.61', '65.03', '95.98'] Mbps
  L

<h1>256x256 Output Model Simulation With Current Generation Hardware</h1>

In [12]:
main(25, 'current', '256')


--- Simulation: Current Gen | Model 256 | Size: 25KB ---

Step: 0
  Bandwidths: ['47.99', '43.43', '53.04'] Mbps
  Latencies:  ['12.61', '15.34', '16.68'] ms
  Server Loads: ['0.00', '0.00', '0.00']
  Queue Lengths: ['0.0', '0.0', '0.0']
  ➤ Agent Action: OFFLOAD to Edge Server 0

Step: 1
  Bandwidths: ['53.82', '47.76', '66.91'] Mbps
  Latencies:  ['15.58', '14.57', '21.91'] ms
  Server Loads: ['0.00', '0.01', '0.01']
  Queue Lengths: ['0.0', '0.0', '0.0']
  ➤ Agent Action: OFFLOAD to Edge Server 0

Step: 2
  Bandwidths: ['24.90', '22.69', '27.62'] Mbps
  Latencies:  ['17.14', '16.96', '12.86'] ms
  Server Loads: ['0.00', '0.01', '0.01']
  Queue Lengths: ['0.0', '0.0', '0.0']
  ➤ Agent Action: OFFLOAD to Edge Server 2

Step: 3
  Bandwidths: ['97.05', '64.11', '39.51'] Mbps
  Latencies:  ['10.82', '21.23', '9.29'] ms
  Server Loads: ['0.00', '0.02', '0.00']
  Queue Lengths: ['0.0', '0.0', '0.0']
  ➤ Agent Action: OFFLOAD to Edge Server 0

Step: 4
  Bandwidths: ['89.69', '45.22', '34.4

<h1>256x256 Output Model Simulation With Next Generation Hardware</h1>

In [13]:
main(25, 'next', '256')


--- Simulation: Next Gen | Model 256 | Size: 25KB ---

Step: 0
  Bandwidths: ['23.37', '99.73', '67.30'] Mbps
  Latencies:  ['5.83', '5.58', '3.09'] ms
  Server Loads: ['0.00', '0.00', '0.00']
  Queue Lengths: ['0.0', '0.0', '0.0']
  ➤ Agent Action: OFFLOAD to Edge Server 0

Step: 1
  Bandwidths: ['49.92', '46.90', '34.34'] Mbps
  Latencies:  ['3.45', '4.24', '5.37'] ms
  Server Loads: ['0.01', '0.00', '0.02']
  Queue Lengths: ['0.0', '0.0', '0.0']
  ➤ Agent Action: OFFLOAD to Edge Server 0

Step: 2
  Bandwidths: ['27.52', '51.04', '91.74'] Mbps
  Latencies:  ['1.89', '3.97', '4.33'] ms
  Server Loads: ['0.00', '0.00', '0.00']
  Queue Lengths: ['0.0', '0.0', '0.0']
  ➤ Agent Action: OFFLOAD to Edge Server 1

Step: 3
  Bandwidths: ['97.04', '67.51', '52.98'] Mbps
  Latencies:  ['4.88', '3.95', '5.94'] ms
  Server Loads: ['0.00', '0.00', '0.00']
  Queue Lengths: ['0.0', '0.0', '0.0']
  ➤ Agent Action: OFFLOAD to Edge Server 2

Step: 4
  Bandwidths: ['114.64', '33.37', '20.79'] Mbps
  La